In [1]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"shreeyash6199","key":"fa0ace82f613ab882fd3fb0b94916db0"}'}

In [0]:
!pip install -q kaggle

In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!kaggle competitions download -c challenges-in-representation-learning-facial-expression-recognition-challenge

  0% 0.00/7.01k [00:00<?, ?B/s]
100% 7.01k/7.01k [00:00<00:00, 4.35MB/s]
 71% 65.0M/92.0M [00:00<00:00, 38.2MB/s]
100% 92.0M/92.0M [00:01<00:00, 92.3MB/s]


In [4]:
!tar -xvf fer2013.tar.gz

fer2013/fer2013.csv
fer2013/README
fer2013/fer2013.bib
fer2013/


In [0]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
df = pd.read_csv('fer2013/fer2013.csv')

In [0]:
lst = []
lst.append([np.array(l.split(),dtype='int') for l in df.pixels[:33000]])
arr=np.array(lst)
X_train = arr.reshape((33000,48,48,1))

In [0]:
lst = []
lst.append([np.array(l.split(),dtype='int') for l in df.pixels[33000:]])
arr=np.array(lst)
X_test = arr.reshape((2887,48,48,1))

In [0]:
emotion = np.array(df.emotion,dtype='int')
emotion_dict = {0:"Angry", 1:"Disgust", 2:"Fear", 3:"Happy", 4:"Sad", 5:"Surprise", 6:"Neutral"}

In [10]:
from keras.utils import to_categorical
Y_train = to_categorical(emotion[:33000],num_classes=7)
Y_test = to_categorical(emotion[33000:],num_classes=7)

Using TensorFlow backend.


In [0]:
import keras
from keras.layers import Conv2D,Dense,MaxPool2D,concatenate,Input,BatchNormalization,Flatten,Activation,AveragePooling2D
from keras.models import Model
from keras.optimizers import  Adam
from keras.layers import add

![ResNet Module](https://raw.githubusercontent.com/raghakot/keras-resnet/master/images/residual_block.png)

In [0]:

def resnet_module(X, modify_dim = False, channels = 64):
    if modify_dim is False:
        X2 = X
    elif modify_dim is True:
        X2 = Conv2D(channels, (1,1),padding='same',strides=1)(X)        
    X1 = BatchNormalization()(X)
    X1 = Activation('relu')(X1)
    X1 = Conv2D(channels, (3,3),padding='same')(X1)
    X1 = BatchNormalization()(X1)
    X1 = Activation('relu')(X1) 
    X1 = Conv2D(channels, (3,3),padding='same')(X1)
    out = Add()([X1,X2])
    return out

    
  

In [0]:
inputs = Input((48,48,1))
X = Conv2D(64,(3,3),padding='same',activation='relu')(inputs)
#====== First resnet module ===============
X = resnet_module(X,False,64)
X = MaxPool2D(2)(X)
#====== 2nd resnet module after reducing size===============
X = resnet_module(X,True,128)
X = MaxPool2D(2)(X)
#====== 3rd resnet module after reducing size===============
X = resnet_module(X,True,256)
#====== average pooling for reducing size===============
X = AveragePooling2D()(X)
#============ Dense layers ================================
X = Flatten()(X)
X = Dense(1024,activation='relu')(X)
out = Dense(7,activation='softmax')(X)



In [0]:
resnet = Model(inputs=inputs,outputs=out)

In [67]:
resnet.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_21 (InputLayer)           (None, 48, 48, 1)    0                                            
__________________________________________________________________________________________________
conv2d_120 (Conv2D)             (None, 48, 48, 64)   640         input_21[0][0]                   
__________________________________________________________________________________________________
batch_normalization_80 (BatchNo (None, 48, 48, 64)   256         conv2d_120[0][0]                 
__________________________________________________________________________________________________
activation_79 (Activation)      (None, 48, 48, 64)   0           batch_normalization_80[0][0]     
__________________________________________________________________________________________________
conv2d_121

In [0]:
adam = Adam(0.001)
resnet.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [0]:
resnet.fit(X_train,Y_train,validation_split = 0.05,epochs=1)

Train on 31350 samples, validate on 1650 samples
Epoch 1/1
 3168/31350 [==>...........................] - ETA: 44:19 - loss: 12.1074 - acc: 0.2462

In [0]:
loss,acc = inception.evaluate(X_test,Y_test)
print('accuracy : ',acc)

2887/2887 [==============================] - 77s 27ms/step
accuracy :  0.24593003120519813


In [0]:
inception.save('inception_faces.h5')
files.download('inception_faces.h5')

----------------------------------------
Exception happened during processing of request from ('::ffff:127.0.0.1', 44540, 0, 0)
Traceback (most recent call last):
  File "/usr/lib/python3.6/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib/python3.6/socketserver.py", line 721, in __init__
    self.handle()
  File "/usr/lib/python3.6/http/server.py", line 418, in handle
    self.handle_one_request()
  File "/usr/lib/python3.6/http/server.py", line 406, in handle_one_request
    method()
  File "/usr/lib/python3.6/http/server.py", line 639, in do_GET
    self.copyfile(f, self.wfile)
  File "/usr/lib/python3.6/http/server.py", line 800, in copyfile
    shutil.copyfil